In [1]:
import os
from langchain_ollama import OllamaLLM
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import DocArrayInMemorySearch # do I need permanent storage like pinecone?
from langchain_ollama import OllamaEmbeddings
from operator import itemgetter


MODEL = "gemma3"
MODEL_EMBED = "embeddinggemma"
DOCUMENTS = "pdfs/resume2025.pdf"

In [2]:
# Instanciate & test model
model = OllamaLLM(model=MODEL)
model.invoke("Don't think of an elephant")

"Okay! I won't think of an elephant. 😉 \n\nHow are you doing?"

In [3]:

#loader = PyPDFLoader(DOCUMENTS)
#pages = loader.load_and_split()

loaders = [PyPDFLoader(os.path.join("pdfs/", fn)) for fn in os.listdir("pdfs/")]
pages = [loader.load_and_split() for loader in loaders]
pages = [pagelet for page in pages for pagelet in page]


In [4]:


template = '''
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
'''

prompt = PromptTemplate.from_template(template)
prompt.format(context="I need a Data Scientist", question="Is this a good fit")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: I need a Data Scientist\n\nQuestion: Is this a good fit\n'

In [5]:
chain = prompt | model 

In [6]:
chain.input_schema.schema()

/var/folders/z2/qtr66ng91gj6czktr43rbl4h0000gn/T/ipykernel_73944/3226659032.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  chain.input_schema.schema()


{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [7]:


embeddings = OllamaEmbeddings(model=MODEL_EMBED)

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [8]:
retriever = vectorstore.as_retriever()

In [9]:


chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | prompt
    | model
    
)

print(chain.invoke({"question": "what soft skills are evident"}))
print(chain.invoke({"question": "would she be a good fit for our data scientist role"}))

Based on the provided documents, here's a breakdown of the soft skills evident in Cindy Reiner’s experience:

*   **Stakeholder Management:** Repeatedly mentioned in her Booz Allen Hamilton role (managing technical operations and stakeholder relationships) and throughout her career.
*   **Cross-functional Leadership:** Demonstrated in her Booz Allen Hamilton role and her experience at Jovial.
*   **Technical Writing & Editing:** Highlighted in her Jovial role and evident in her resume itself.
*   **Grant Writing:** Mentioned as a skill.
*   **Presentations:** Demonstrated in her Data Science Bootcamp and evident in her resume.
*   **Communication:** Implicit in her roles requiring interactions with clients, colleagues, and stakeholders.
*   **Teamwork/Collaboration:** Evident in her work on the Capstone Project at Flatiron School and her collaboration at Booz Allen Hamilton.
*   **Problem-Solving:** Suggested by her design and implementation of data processing pipelines and classificat

In [10]:
print(chain.invoke({"question": "what are the main takeaways from her research"}))

Based on the provided documents, here’s a breakdown of Cindy Reiner’s key research takeaways:

*   **Protein-Protein Interactions & Signaling Pathways:** Her primary research focused on investigating protein-protein interactions and cellular signaling pathways, particularly those involving small GTPases (Rab proteins).
*   **mAChRs and Receptor-G-Protein Interactions:** Specifically, she studied the interactions between muscarinic acetylcholine receptors (mAChRs) and their associated G-proteins.
*   **Novel Receptor Identification:** She identified a novel RACK1-M2 receptor interaction.
*   **Rab Protein Function:** She explored the functions of Rab11, Rab15, and Rab22, investigating their roles in cellular processes.
*   **Technical Expertise:** She demonstrated proficiency in a wide range of techniques including cell culture, transfection, radioligand binding assays, and data analysis.

In essence, Cindy Reiner’s research centered on elucidating the molecular mechanisms governing cel